In [ ]:
#非即時
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
import librosa

def sound(window,s,m):

    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) >= 500: #判斷S的總和是否>=500，如果>=500，代表有明顯的聲音
        window = window.astype(np.float)

        window = (window - window.mean()) / (window.max() - window.min())

        features = python_speech_features.base.logfbank(window,
                                                        samplerate=16000,
                                                        winlen=0.025,
                                                        winstep=0.01,
                                                        nfilt=26,
                                                        nfft=512,
                                                        lowfreq=0,
                                                        highfreq=None,
                                                        preemph=0.97)
        
        # Make prediction from model
        in_tensor = np.float32(features.reshape(1, features.shape[0], features.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max > 0.7):
            print(words[list_val_maxIndex] + " " + str(m) + "分" + str(s) + "秒" + " 預測值:" + str(list_val_max))#輸出相對應的字詞
        
#main
# Parameters
model_path = './tflite_normalize/recording9_fbank.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '其他', '吼', '啦', '嗯', '好', '我', '的', '著', '那', '阿']

data = []
start = 0 #一開始的索引值
end = 4000 #一開始的索引值
s = 0 #秒
m = 0 #分
sample_rate = 16000 #取樣率

#載入音檔
y, sr = librosa.load("./TestWAV/TEST.wav",sr=sample_rate) 

#用0填滿時間，以讓滑動視窗都有東西
total_sec = len(y) / 16000 #算音檔總時間
if total_sec < 1: #如果音檔<1s
    new_y = np.zeros(16000) #new_y補足1s
    new_y[0:len(y)] = y #將原先的音訊載入到新的陣列
elif total_sec - (int(total_sec)) != 0: #如果音檔非整數秒，ex:原先音訊長度1.2s ，1.2 - 1 != 0 ， 將長度補到2s
    total_sec = (int(total_sec) + 1) #新時間為原時間整數+1
    new_y = np.zeros(total_sec * 16000) #補足到整數秒
    new_y[0:len(y)] = y #將原先的音訊載入到新的陣列
else:
    new_y = y

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

while True:
    s = s + 0.25 #增加秒數
    if(s >= 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    window[:4000] = window[4000:] #把音訊載入window
    window[4000:] = new_y[start:end] #把音訊載入window
    
    sound(window,s,m) #呼叫sound()

    if(end == len(new_y)): #如果移動到最後，break
        break
    
    start = start + 4000 #向後移動
    end = end + 4000 #向後移動    